## Thought Process
### Step 1: Gather Segmentations
a. Define "Get" commands for each all images that will be pre-stored in a folder (similar to get_nuclei) 

b. Define "Get" command that compiles all images into a list while applying cellmask

### Step 2: Combine Segmentations
a. Ensure all segmented variables are set to type uint16

b. Add all segmented variables together into one matrix

### Step 3: Segment by values
a. Determine the highest value

b. separate the combo into values that are >=2, >=3, >=4, etc.

c. Assign each segmented value a label with the corresponding labeled objects from the segmented images

## Imports

In [ ]:
# top level imports
from pathlib import Path
from collections import Counter
import os, sys
import numpy as np
import copy
from skimage.segmentation import *
from skimage.measure import label

# # package for io 
import napari

### import local python functions in ../infer_subc
sys.path.append(os.path.abspath((os.path.join(os.getcwd(), '..'))))

from infer_subc.core.file_io import (read_czi_image,
                                     import_inferred_organelle,
                                     export_inferred_organelle,
                                     list_image_files)

from infer_subc.core.img import *

LD_CH = 0
NUC_CH = 1
LYSO_CH = 2
MITO_CH = 3
GOLGI_CH = 4
PEROX_CH = 5
ER_CH = 6
PM_CH = 7
RESIDUAL_CH = 8
TEST_IMG_N = 1
ORGANELLES = ["LD",
              "nuclei", 
              "lyso", 
              "mito", 
              "golgi", 
              "perox", 
              "ER",
              ]

In [ ]:
viewer = napari.Viewer()

## Get & Load Image for Processing

In [ ]:
test_img_n = TEST_IMG_N

data_root_path = Path(os.path.expanduser("~")) / "Documents/Python Scripts/Infer-subc-2D"

in_data_path = data_root_path / "raw mcz"
im_type = ".tiff"

img_file_list = list_image_files(in_data_path,im_type)
test_img_name = img_file_list[test_img_n]

out_data_path = data_root_path / "segmented"
if not Path.exists(out_data_path):
    Path.mkdir(out_data_path)
    print(f"making {out_data_path}")

In [ ]:
img_data,meta_dict = read_czi_image(test_img_name) #read_czi_image works for tiff images as well

channel_names = meta_dict['name']
img = meta_dict['metadata']['aicsimage']
scale = meta_dict['scale']
channel_axis = meta_dict['channel_axis']

In [ ]:
viewer.add_image(img_data, scale=scale)

## Get functions

Generic

In [ ]:
#Generic get organelle function--collects the organelle, and the metadata for the organelle, returns organelle
def get_organelle(name: str, meta_dict: dict, out_data_path: Path) -> np.ndarray:
    """
    load organelle if it exists, otherwise return None
    
    Parameters
    ------------
    name:
        a string of the name used to identify the organelle in the files
    meta_dict:
        dictionary of meta-data (ome)
    out_data_path:
        Path object where tiffs are written to

    Returns
    -------------
    exported file name

    """
    try:                                                                        #if there is no error, proceed
        organelle = import_inferred_organelle(name, meta_dict, out_data_path)   #imports the organelle
    except:                                                                     #if there is an error, proceed
        print("cannot find organelle, returning None")                          #prints that it cannot find the organelle
        organelle = None                                                        #sets organelle value to None
    return organelle                                                            #returns the organelle data

Organelle List

In [ ]:
#collects segmented organelles and puts them into a binary and a labeled dictionary using organelle inputs
def get_organelles(meta_dict:dict, out_data_path:Path, organelles:list[str]) -> dict:
    labels_organelles={}                                                                                    #list for storing labeled organelles
    binary_organelles={}                                                                                    #list for storing binary organelles
    cellmask=get_organelle("cell",meta_dict,out_data_path)                                                  #collects the cellmask
    for i in range(len(organelles)):                                                                        #iterates across each organelle in organelles input
        labels_organelles[organelles[i]]=((cellmask>0)*get_organelle(organelles[i],meta_dict,out_data_path))#adds collected organelle data to labeled organelles
        binary_organelles[organelles[i]]=((labels_organelles[organelles[i]]>0))                             #unlabels the collected organelle data and adds it to labels_binary
    return labels_organelles,binary_organelles                                                              #returns the organelle lists

## Combine Segmentations

In [ ]:
labels_org, binary_org = get_organelles(meta_dict, out_data_path, ORGANELLES)

## Multiple Contact Prototype:

In [ ]:
#checks if a string can be turned into a list that is equal to any preexisting lists in an input dictionary
def inkeys(dic: dict[str], s:str) -> bool:
    a = sorted(s.split("_"))                #Normalizes the way the string is written
    for key in dic.keys():                  #Iterates through each key in dictionary
        if (a == sorted(key.split("_"))):   #Checks if key has same vals as string
            return True                     #Returns true if any key is same as string
    return False                            #Returns false if no keys are same as string

In [ ]:
#Finds contact sites between 2 organelles and assigns them to a dictionary, and returns the dictionary
def two_contact(binary: dict[str], contact:dict[str] = {}) -> dict:
    for a in (binary.keys()):                              #Iterates across all labeled organelle images (a)
        for b in (binary.keys()):                          #Iterates across all labeled organelle images (b)
            b_a=b+"_"+a                                    #Creates string to check for found a & b contact
            a_b=a+"_"+b                                    #Creates new potential key for a & b contact
            if ((a!=b) and not                             #Ensures a is not the same as b
                (b_a in contact.keys()) and                #Ensures no contacts of a & b are already found
                np.any((binary[a]*binary[b]))):            #Ensures contact is between a & b
                contact[a_b]=(label(binary[a]*binary[b]))  #Assigns contact of a & b to dictionary
    return contact                                         #Returns the dictionary

In [ ]:
#Finds areas in contact sites between 2 or more organelles that also have contacts with an additional organelle
#and assigns those areas as new contact sites to a dictionary, and returns the dictionary. 
#Works for contacts of 3 or more organelles
def contacting(binary: dict[str], iterated:dict[str]) -> dict: 
    contact={}                                                      #Creates empty dictionary
    for c in iterated:                                              #Iterates through preexisting dictionary of contact sites
        for b in binary:                                            #Iterates through labeled organelles list (b)
            if(np.any((iterated[c]*binary[b])>0) and not            #Proceeds if there are contacts present 
               ((b+"_" in c)or("_"+b in c)or("_"+b+"_" in c)) and   #Proceeds if organelle is not already in previous contact set
               (not inkeys(contact,(c+"_"+b)))):                    #Proceeds if contact between organelle and previous contact set is not already made  
                contact[(c+"_"+b)]=((iterated[c]*binary[b])>0)      #Adds new binary contact
    return contact                                                  #Returns contacts

In [ ]:
#Assigns all contacts of 2 or more organelles to a dictionary, and returns the dictionary
def multi_contact(binary:dict[str:np.ndarray], 
                  organelles:list[str]) -> dict:
    contacts=two_contact(binary)                #Dictionary for ALL contacts
    iterated={}                                 #Iterated Dictionary
    contact={}                                  #Contact Dictionary

    #Repeat # of times equal to max # of contacts in one contact site
    for n in (range(len(organelles)-1)): 
        iterated.clear()                        #Clears iterated dictionary
        contact.clear()                         #Clears contact dictionary
        num=n+3                                 #Number of contacts
        for key in contacts:                    #Iterates over every key in contacts
            if (len(key.split("_")) == (num-1)):#Selects for last set of contacts
                iterated[key]=contacts[key]     #Adds each key from last set of contacts to iterated
        contact=contacting(binary,iterated)     #Collects higher order contact sites
        for d in contact:                       #Iterates across the new contact sites 
            contacts[d]=label(contact[d])       #Labels & assigns higher order contact to dictionary
    return contacts                             #Returns with dictionary of all levels of contacts

In [ ]:
#Organizing by sites
def multi_contact_2(contacts:dict[str:np.ndarray], 
                    organelles:list[str]) -> dict:
    HO = {}                                             #Higher Order Contacts
    LO = {}                                             #Lower Order Contacts
    interactions = copy.deepcopy(contacts)              #Export dictionary
    for n in range(len(organelles)-1):                                              #Repeat # of times equal to max # of contacts in one contact site
        HO.clear()
        LO.clear()
        num = n+3                                       #Number of organelles in contact at site

                                                        #Collects all higher and lower order interactions
        for key in interactions:
            if (len(key.split("_"))==(num)):
                HO[key]=interactions[key]
            if (len(key.split("_"))==(num-1)):
                LO[key]=interactions[key]

        #Iterates across each higher order interaction
        for HO_key in HO:
            HO_orgs = HO_key.split("_")                                             #Creates a list of organelles present in higher order interaction
            #Iterates across each lower order interaction
            for LO_key in LO:
                if set(LO_key.split("_")).issubset(HO_orgs):                        #Selects for organelles with LO interactions present in HO interaction
                    ws = watershed(LO[LO_key],                                      #Selects for specific LO interactions with HO interactions
                                   markers=HO[HO_key],
                                   connectivity=np.ones((3, 3, 3), bool),
                                   mask=LO[LO_key])
                    #apply mask function?
                    removal = np.ones_like(ws)
                    removal[ws >= 1] = 0
                    removed = LO[LO_key] * removal                          #Removes LO interaction sites with HO interactions present
                    interactions.update(LO_key=removed)                     #edits the interactions LO data
    return interactions

In [ ]:
mc = multi_contact(binary_org, ORGANELLES)

In [ ]:
mc2 = multi_contact_2(copy.deepcopy(mc), ORGANELLES)

In [ ]:
interaction = "lyso_golgi_ER"
viewer.add_labels(mc[interaction], name=("MC: ")+interaction)
viewer.add_labels(mc2[interaction], name=("MC2: ")+interaction)

In [ ]:
#Returns # of images with desired organelle in contact with at least one other
#Has the option to add each image to napari for visualizing
def get_all_organelle_contact(labeled:dict[str], organelle: str, view:bool=False, v=napari.Viewer()) -> dict:
    lo = {}                                            #Initializes a dictionary with desired organelle
    for key in labeled:                                #Iterates across all contact images
        if (organelle in key):                         #proceeds with all images that match organelle
            lo[key] = labeled[key]                     #adds image to dictionary
            if view:                                   #checks if adding to viewer is desired
                v.add_labels(labeled[key], name=key)   #adds those images to napari viewer
    return lo                                          #Returns dictionary

In [ ]:
print(mc.keys())

In [ ]:
viewer.add_labels(labels_org["perox"])